In [2]:
import torch

a = torch.zeros((10, 10))

t = torch.tensor([[0, 1, 6, 9]])

a[t] = 1
a

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [3]:
feature_extractor = torch.hub.load('facebookresearch/dinov2',
                                                    'dinov2_vitb14')

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /home/barti/.cache/torch/hub/main.zip
xFormers not available
xFormers not available
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /home/barti/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:27<00:00, 12.7MB/s] 


In [10]:
dummy = torch.randn((4, 3, 224, 224))

d = feature_extractor.forward(dummy)

d.shape

torch.Size([4, 768])

In [24]:
# 5x5 tensor with values from 0 to 24
x = torch.arange(25).reshape(5, 5)

s = torch.tensor([1, 3])

print(x.shape, s.shape)
print(x, s)

x[s]

torch.Size([5, 5]) torch.Size([2])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]]) tensor([1, 3])


tensor([[ 5,  6,  7,  8,  9],
        [15, 16, 17, 18, 19]])

In [38]:
x - (x - 2)

tensor([[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]])

In [36]:
#distances between two vectors
a = torch.tensor([[1, 2, 3],[6, 7, 8]], dtype=torch.float32)
b = torch.tensor([[4, 5, 6],[1, 2, 3]], dtype=torch.float32)

print(a)
print(b)

torch.cdist(a, b)[:, :-1], (9 + 9 + 9)**0.5, (25 + 25 + 25)**0.5

tensor([[1., 2., 3.],
        [6., 7., 8.]])
tensor([[4., 5., 6.],
        [1., 2., 3.]])


(tensor([[5.1962],
         [3.4641]]),
 5.196152422706632,
 8.660254037844387)

In [62]:
a = torch.randn(32, 10)
b = torch.randn(64, 10)

# Expand tensor a to match the shape of b
# Shape: (32, 1, 10)
expanded_a = a.unsqueeze(1).expand(-1, b.shape[0], -1)  # Shape: (32, 64, 10)

# Expand tensor b to match the shape of a
# Shape: (1, 64, 10)
expanded_b = b.unsqueeze(0).expand(a.shape[0], -1, -1)  # Shape: (32, 64, 10)

# Calculate the Euclidean distance
distance = torch.sqrt(torch.sum((expanded_a - expanded_b) ** 2, dim=-1))  # Shape: (32, 64)

torch.all(distance[0] == euclidean_distance(a[0], b))

distance.argmin(dim=-1, keepdim=True).shape

torch.Size([32, 1])

In [70]:
def euclidean_distance(x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    """
    Computes the euclidean distance between two tensors.

    Args:
        x1 (torch.Tensor): First input tensor.
        x2 (torch.Tensor): Second input tensor.

    Returns:
        torch.Tensor: Euclidean distance between the two tensors of shape (batch_size).
    """
    # assert x1.shape == x2.shape, 'x1 and x2 must have the same shape'
    if x1.shape == x2.shape or any([x1.shape[0] == 1, x2.shape[0] == 1]):
        return (x1 - x2).pow_(2).sum(dim=-1).sqrt_()
    # Expand tensor x1 to match the shape of x2
    # Shape: (32, 1, 10)
    expanded_x1 = x1.unsqueeze(1).expand(-1, x2.shape[0], -1)

    # Expand tensor x2 to match the shape of x1
    # Shape: (1, 64, 10)
    expanded_x2 = x2.unsqueeze(0).expand(x1.shape[0], -1, -1)

    return torch.sqrt(torch.sum((expanded_x1 - expanded_x2) ** 2, dim=-1))

euclidean_distance(a[0].unsqueeze(0), b) == euclidean_distance(a, b)[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [133]:
predictions = euclidean_distance(a, b).argmin(dim=-1)

# tensor of shape 32 with random numbers from 0 to 64
y = torch.randint(0, 64, (32, ))

(predictions == y).sum().float() / y.shape[0]


tensor(0.0625)

In [17]:
zz = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=torch.float).mean(dim=0)

dd = torch.zeros((10, 4))
dd[3] = zz
dd

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [3., 4., 5., 6.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [7]:
d['x_norm_patchtokens'].shape

torch.Size([1, 256, 768])

In [8]:
torch.cat([d['x_norm_clstoken'], d['x_norm_patchtokens'].mean(dim=1)], dim=-1).shape

torch.Size([1, 1536])